In [26]:
# https://kanoki.org/2019/02/26/compare-two-excel-files-for-difference-using-python/

# Stock trade verification code

import numpy as np
import pandas as pd

In [27]:
oms_df = pd.read_excel("C:/PythonProjects/data/20230228_oms.xls", )
trader_df = pd.read_excel("C:/PythonProjects/data/2월28일 거래.xlsx", )

In [28]:
oms_df

,No,종목코드,종목명,매매유형,매매구분,실주문수량,단가,체결수량,체결금액,체결단가,체결잔량,현재가,금액,비율,Unnamed: 14,Unnamed: 15,STP,FIX
0,1,KR7009150004,삼성전기,차입,매도,240,147300,240,34943200,145597,0,144000,-2900,-1.97,-2.15,1.11,100,100
1,2,KR7066970005,엘앤에프,차입,매수,920,241000,920,252881000,274871,0,262000,21500,8.94,7.28,-4.68,100,100
2,3,KR7078600004,대주전자재료,차입,매도,230,94700,230,21743000,94535,0,95300,2300,2.47,0.75,-0.80,100,100
3,4,KR7098460009,고영,차입,매수,5630,16640,5630,94130710,16719,0,16750,110,0.66,0.45,0.18,100,100
4,5,KR7161390000,한국타이어앤테크놀로지,차입,매도,9200,37450,9200,346873050,37704,0,37200,-250,-0.67,-0.82,1.35,100,100
5,6,KR7168360006,펨트론,일반,매수,1000,8250,1000,8233300,8233,0,8200,-150,-1.80,-1.02,-0.40,100,100
6,7,KR7360070007,탑머티리얼,일반,매도,300,39500,300,12000000,40000,0,38850,250,0.65,-0.41,2.96,100,100
7,8,KR7377300009,카카오페이,차입,매도,3780,62000,3780,234157300,61946,0,62700,1800,2.96,0.47,-1.20,100,100


In [29]:
trader_df

,펀드,매매처,단축코드,종목명,매매구분,체결수량,체결단가,체결금액
0,공모주1호,NH,66970.0,엘앤에프,Buy cover,270.0,273500.0,73845000.0
1,공모주2호,Mirae,366070.0,탑머티리얼,Sell,300.0,40000.0,12000000.0
2,멀티전략,KIS,78600.0,대주전자재료,Short sell,70.0,94523.0,-6616610.0
3,멀티전략,KIS,168360.0,펨트론,Buy,1000.0,8233.0,-8233000.0
4,멀티전략,KIS,9150.0,삼성전기,Short sell,70.0,145654.0,-10195780.0
5,멀티전략,KIS,66970.0,엘앤에프,Buy cover,650.0,275440.0,179036000.0
6,멀티전략,KIS,98460.0,고영,Buy cover,3173.0,16715.0,53036695.0
7,멀티전략,KIS,161390.0,한국타이어앤테크놀로지,Short sell,2400.0,37712.0,-90508800.0
8,멀티전략,KIS,98460.0,고영,Buy cover,2457.0,16725.0,41093325.0
9,코스닥벤처,Yuanta,161390.0,한국타이어앤테크놀로지,Short sell,6800.0,37701.0,-256366800.0


In [30]:
# https://sparkbyexamples.com/pandas/pandas-select-columns-by-name-or-index/
# 필요한 열 선택

oms_df1 = oms_df[["종목명", "매매유형", "매매구분", "체결수량", "체결단가", "체결금액"]]

In [31]:
oms_df1

,종목명,매매유형,매매구분,체결수량,체결단가,체결금액
0,삼성전기,차입,매도,240,145597,34943200
1,엘앤에프,차입,매수,920,274871,252881000
2,대주전자재료,차입,매도,230,94535,21743000
3,고영,차입,매수,5630,16719,94130710
4,한국타이어앤테크놀로지,차입,매도,9200,37704,346873050
5,펨트론,일반,매수,1000,8233,8233300
6,탑머티리얼,일반,매도,300,40000,12000000
7,카카오페이,차입,매도,3780,61946,234157300


In [32]:
# Short sell, Buy cover, Buy, Sell 매매구분을 위한 열 병합 및 값 변환
oms_df1["매매구분"] = oms_df1[["매매유형", "매매구분"]].agg('_'.join, axis=1)

oms_df1["매매구분"] = oms_df1["매매구분"].replace(['일반_매수', '일반_매도', '차입_매수', '차입_매도'], ['Buy', 'Sell', 'Buy cover', 'Short sell'])

oms_df1 = oms_df1.drop(oms_df1.columns[[1]], axis=1)

oms_df1 = oms_df1.replace(0, np.nan)
oms_df1 = oms_df1.dropna()

col_to_move = oms_df1.pop('매매구분')

oms_df1.insert(1, '매매구분', col_to_move)

oms_df1

C:\Users\jtlee\AppData\Local\Temp\ipykernel_32480\644846331.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oms_df1["매매구분"] = oms_df1[["매매유형", "매매구분"]].agg('_'.join, axis=1)
C:\Users\jtlee\AppData\Local\Temp\ipykernel_32480\644846331.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oms_df1["매매구분"] = oms_df1["매매구분"].replace(['일반_매수', '일반_매도', '차입_매수', '차입_매도'], ['Buy', 'Sell', 'Buy cover', 'Short sell'])


,종목명,매매구분,체결수량,체결단가,체결금액
0,삼성전기,Short sell,240,145597,34943200
1,엘앤에프,Buy cover,920,274871,252881000
2,대주전자재료,Short sell,230,94535,21743000
3,고영,Buy cover,5630,16719,94130710
4,한국타이어앤테크놀로지,Short sell,9200,37704,346873050
5,펨트론,Buy,1000,8233,8233300
6,탑머티리얼,Sell,300,40000,12000000
7,카카오페이,Short sell,3780,61946,234157300


#둘 다 키값을 종목명(Key1) and 체결수량(Key2)으로 잡고 정렬한 다음 데이터프레임의 밸류들이 일치하는지 확인

In [33]:
oms_df1

,종목명,매매구분,체결수량,체결단가,체결금액
0,삼성전기,Short sell,240,145597,34943200
1,엘앤에프,Buy cover,920,274871,252881000
2,대주전자재료,Short sell,230,94535,21743000
3,고영,Buy cover,5630,16719,94130710
4,한국타이어앤테크놀로지,Short sell,9200,37704,346873050
5,펨트론,Buy,1000,8233,8233300
6,탑머티리얼,Sell,300,40000,12000000
7,카카오페이,Short sell,3780,61946,234157300


In [34]:
oms_df2 = oms_df1.set_index('종목명')

oms_df3 = oms_df2.sort_index(axis=0)

In [35]:
oms_df3

,매매구분,체결수량,체결단가,체결금액
종목명,,,,
고영,Buy cover,5630,16719,94130710
대주전자재료,Short sell,230,94535,21743000
삼성전기,Short sell,240,145597,34943200
엘앤에프,Buy cover,920,274871,252881000
카카오페이,Short sell,3780,61946,234157300
탑머티리얼,Sell,300,40000,12000000
펨트론,Buy,1000,8233,8233300
한국타이어앤테크놀로지,Short sell,9200,37704,346873050


In [36]:
trader_df1 = trader_df[["종목명", "매매구분", "체결수량", "체결단가", "체결금액"]]

In [37]:
trader_df1 = trader_df1.dropna()

trader_df2 = trader_df1.set_index('종목명')

col_to_move2 = trader_df2.pop('매매구분')
price_to_move = trader_df2.pop('체결단가')

col_to_move2 = col_to_move2.reset_index()
price_to_move = price_to_move.reset_index()

col_to_move3 = col_to_move2.drop_duplicates(subset=['종목명'], keep='first')
price_to_move2 = price_to_move.drop_duplicates(subset=['종목명'], keep='first')

col_to_move3 = col_to_move3.set_index('종목명')
price_to_move2 = price_to_move2.set_index('종목명')

trader_df2 = trader_df1.groupby(trader_df1['종목명'], sort=False).sum()

trader_df2 = trader_df2.drop('체결단가', axis=1)

# trader_df2 = trader_df2.reset_index()
 
trader_df3 = trader_df2.merge(col_to_move3, how='outer', left_index=True, right_index=True)
trader_df3 = trader_df3.merge(price_to_move2, how='outer', left_index=True, right_index=True)

trader_df3 = trader_df3[['매매구분', '체결수량', '체결단가', '체결금액']]

# trader_df2 = trader_df1.set_index('종목명')

# trader_df3 = trader_df2.sort_index(axis=0)

C:\Users\jtlee\AppData\Local\Temp\ipykernel_32480\2975992591.py:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  trader_df2 = trader_df1.groupby(trader_df1['종목명'], sort=False).sum()


In [38]:
trader_df3

,매매구분,체결수량,체결단가,체결금액
종목명,,,,
엘앤에프,Buy cover,920.0,273500.0,252881000.0
탑머티리얼,Sell,300.0,40000.0,12000000.0
대주전자재료,Short sell,230.0,94523.0,-21743010.0
펨트론,Buy,1000.0,8233.0,-8233000.0
삼성전기,Short sell,240.0,145654.0,-34943190.0
고영,Buy cover,5630.0,16715.0,94130020.0
한국타이어앤테크놀로지,Short sell,9200.0,37712.0,-346875600.0
카카오페이,Short sell,3780.0,61920.0,-234156400.0


In [39]:
# col_to_move2 = col_to_move2.reset_index()
col_to_move3 = col_to_move2.drop_duplicates(subset=['종목명'], keep='first')

In [40]:
col_to_move3

,종목명,매매구분
0,엘앤에프,Buy cover
1,탑머티리얼,Sell
2,대주전자재료,Short sell
3,펨트론,Buy
4,삼성전기,Short sell
6,고영,Buy cover
7,한국타이어앤테크놀로지,Short sell
12,카카오페이,Short sell


In [41]:
trader_df3 = trader_df3.sort_index(axis=0)

In [42]:
trader_df3 = trader_df3.astype({'체결수량' : 'int', '체결단가' : 'int', '체결금액' : 'int'})

In [43]:
trader_df3['체결금액'] = trader_df3['체결금액'].abs()

# try:
#     trader_df3 = trader_df3.drop(['코스닥150F', '코스피200F', '코스닥F150', '코스피F200'])
# except:
#     pass

try:
    trader_df3 = trader_df3.drop(['코스닥150F'])
except:
    pass
try:
    trader_df3 = trader_df3.drop(['코스피200F'])
except:
    pass
try:
    trader_df3 = trader_df3.drop(['코스닥F150'])
except:
    pass
try:
    trader_df3 = trader_df3.drop(['코스피F200'])
except:
    pass

In [44]:
# trader_df3.compare(oms_df3)

In [45]:
trader_df3 = trader_df3.reset_index()
oms_df3 = oms_df3.reset_index()

In [46]:
trader_df3

,종목명,매매구분,체결수량,체결단가,체결금액
0,고영,Buy cover,5630,16715,94130020
1,대주전자재료,Short sell,230,94523,21743010
2,삼성전기,Short sell,240,145654,34943190
3,엘앤에프,Buy cover,920,273500,252881000
4,카카오페이,Short sell,3780,61920,234156400
5,탑머티리얼,Sell,300,40000,12000000
6,펨트론,Buy,1000,8233,8233000
7,한국타이어앤테크놀로지,Short sell,9200,37712,346875600


In [47]:
oms_df3

,종목명,매매구분,체결수량,체결단가,체결금액
0,고영,Buy cover,5630,16719,94130710
1,대주전자재료,Short sell,230,94535,21743000
2,삼성전기,Short sell,240,145597,34943200
3,엘앤에프,Buy cover,920,274871,252881000
4,카카오페이,Short sell,3780,61946,234157300
5,탑머티리얼,Sell,300,40000,12000000
6,펨트론,Buy,1000,8233,8233300
7,한국타이어앤테크놀로지,Short sell,9200,37704,346873050


In [48]:
result_df = oms_df3

In [49]:
result_df[['종목명']] = oms_df3[['종목명']]
result_df[['매매구분']] = oms_df3[['매매구분']]
result_df[['체결수량']] = oms_df3[['체결수량']] - trader_df3[['체결수량']]
result_df[['체결단가']] = oms_df3[['체결단가']] - trader_df3[['체결단가']]
result_df[['체결금액']] = oms_df3[['체결금액']] - trader_df3[['체결금액']]

In [50]:
result_df[['종목명', '매매구분', '체결단가', '체결수량', '체결금액']]

,종목명,매매구분,체결단가,체결수량,체결금액
0,고영,Buy cover,4,0,690
1,대주전자재료,Short sell,12,0,-10
2,삼성전기,Short sell,-57,0,10
3,엘앤에프,Buy cover,1371,0,0
4,카카오페이,Short sell,26,0,900
5,탑머티리얼,Sell,0,0,0
6,펨트론,Buy,0,0,300
7,한국타이어앤테크놀로지,Short sell,-8,0,-2550
